In [14]:
import pandas as pd
import numpy as np

# 显示全部结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## 一、Patsy

**更多参考：** https://patsy.readthedocs.io/

patsy是一个用于描述性统计模型（尤其是线性模型）的python库。

In [66]:
import patsy

### （一）基本描述

In [67]:
# 生成数据
df = pd.DataFrame({
    'x_0': [1, 2, 3, 4, 5],
    'x_1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

df

,x_0,x_1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [68]:
# 不加截距
y, X = patsy.dmatrices('y ~ x_0 + x_1 + 0', df)
y
X

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

DesignMatrix with shape (5, 2)
  x_0    x_1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
  Terms:
    'x_0' (column 0)
    'x_1' (column 1)

In [69]:
# 加截距
y, X = patsy.dmatrices('y ~ x_0 + x_1', df)
y
X

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

DesignMatrix with shape (5, 3)
  Intercept  x_0    x_1
          1    1   0.01
          1    2  -0.01
          1    3   0.25
          1    4  -4.10
          1    5   0.00
  Terms:
    'Intercept' (column 0)
    'x_0' (column 1)
    'x_1' (column 2)

In [70]:
# 最小二成回归
coef, resid, _, _ = np.linalg.lstsq(X,y)
coef
v = coef.squeeze()
index_names = X.design_info.column_names
coef = pd.Series(v, index=index_names)
coef

D:\software\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

Intercept    0.312910
x_0         -0.079106
x_1         -0.265464
dtype: float64

### （二）数据转换

In [71]:
# 生成数据
df = pd.DataFrame({
    'x_0': [1, 2, 3, 4, 5],
    'x_1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

df

,x_0,x_1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [72]:
y, X = patsy.dmatrices('y ~ x_0 + np.log(np.abs(x_1) + 1)', df)
y
X

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

DesignMatrix with shape (5, 3)
  Intercept  x_0  np.log(np.abs(x_1) + 1)
          1    1                  0.00995
          1    2                  0.00995
          1    3                  0.22314
          1    4                  1.62924
          1    5                  0.00000
  Terms:
    'Intercept' (column 0)
    'x_0' (column 1)
    'np.log(np.abs(x_1) + 1)' (column 2)

In [73]:
y, X = patsy.dmatrices('y ~ standardize(x_0) + center(x_1)', df)
y
X

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

DesignMatrix with shape (5, 3)
  Intercept  standardize(x_0)  center(x_1)
          1          -1.41421         0.78
          1          -0.70711         0.76
          1           0.00000         1.02
          1           0.70711        -3.33
          1           1.41421         0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x_0)' (column 1)
    'center(x_1)' (column 2)

利用 patsy.build_design_matrices 函数将原始样本内数据集中保存的信息应用到新的数据集上，也可以看作一种映射吧。

In [74]:
new_df = pd.DataFrame({
    'x_0': [6, 7, 8, 9, 10],
    'x_1': [0.02, -0.02, 0.50, -8.2, 0.],
    'y': [-3., 0., 7.2, 2.6, -4.]})

new_df

,x_0,x_1,y
0,6,0.02,-3.0
1,7,-0.02,0.0
2,8,0.50,7.2
3,9,-8.20,2.6
4,10,0.00,-4.0


In [75]:
new_X = patsy.build_design_matrices([X.design_info], new_df)
new_X

[DesignMatrix with shape (5, 3)
   Intercept  standardize(x_0)  center(x_1)
           1           2.12132         0.79
           1           2.82843         0.75
           1           3.53553         1.27
           1           4.24264        -7.43
           1           4.94975         0.77
   Terms:
     'Intercept' (column 0)
     'standardize(x_0)' (column 1)
     'center(x_1)' (column 2)]

In [77]:
# 两列相加，要用I函数
y, X = patsy.dmatrices('y ~ I(x_0 + x_1)', df)
y
X

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

DesignMatrix with shape (5, 2)
  Intercept  I(x_0 + x_1)
          1          1.01
          1          1.99
          1          3.25
          1         -0.10
          1          5.00
  Terms:
    'Intercept' (column 0)
    'I(x_0 + x_1)' (column 1)

### （三）分类数据

在patsy公式中，使用非数字名词列时，将会默认转换成虚拟变量

In [78]:
df = pd.DataFrame({'key_1': ['a', 'a', 'b', 'b', 'a'],
                 'key_2': [0, 1, 1, 0, 1],
                 'v1': [1, 2, 3, 4, 5],
                 'v2': [5, 4, 3, 2, 1]})
df

,key_1,key_2,v1,v2
0,a,0,1,5
1,a,1,2,4
2,b,1,3,3
3,b,0,4,2
4,a,1,5,1


In [79]:
y, X = patsy.dmatrices('v2 ~ key_1', df)
y
X

DesignMatrix with shape (5, 1)
  v2
   5
   4
   3
   2
   1
  Terms:
    'v2' (column 0)

DesignMatrix with shape (5, 2)
  Intercept  key_1[T.b]
          1           0
          1           0
          1           1
          1           1
          1           0
  Terms:
    'Intercept' (column 0)
    'key_1' (column 1)

In [80]:
# 忽略截距的情况下，每个分类变量都会被单独列出来并赋值
y, X = patsy.dmatrices('v2 ~ key_1 + 0', df)
y
X

DesignMatrix with shape (5, 1)
  v2
   5
   4
   3
   2
   1
  Terms:
    'v2' (column 0)

DesignMatrix with shape (5, 2)
  key_1[a]  key_1[b]
         1         0
         1         0
         0         1
         0         1
         1         0
  Terms:
    'key_1' (columns 0:2)

In [81]:
# 对于数值型可以用C函数转换成分类函数
y, X = patsy.dmatrices('v2 ~ C(key_2)', df)
y
X

DesignMatrix with shape (5, 1)
  v2
   5
   4
   3
   2
   1
  Terms:
    'v2' (column 0)

DesignMatrix with shape (5, 2)
  Intercept  C(key_2)[T.1]
          1              0
          1              1
          1              1
          1              0
          1              1
  Terms:
    'Intercept' (column 0)
    'C(key_2)' (column 1)

In [82]:
# 多列分类数据
df['key_2'] = df['key_2'].map({0: 'zero', 1: 'one'})
df

,key_1,key_2,v1,v2
0,a,zero,1,5
1,a,one,2,4
2,b,one,3,3
3,b,zero,4,2
4,a,one,5,1


In [83]:
y, X = patsy.dmatrices('v2 ~ key_1 + key_2', df)
y
X

DesignMatrix with shape (5, 1)
  v2
   5
   4
   3
   2
   1
  Terms:
    'v2' (column 0)

DesignMatrix with shape (5, 3)
  Intercept  key_1[T.b]  key_2[T.zero]
          1           0              1
          1           0              0
          1           1              0
          1           1              1
          1           0              0
  Terms:
    'Intercept' (column 0)
    'key_1' (column 1)
    'key_2' (column 2)

In [84]:
y, X = patsy.dmatrices('v2 ~ key_1 + key_2 + key_1:key_2', df)
y
X

DesignMatrix with shape (5, 1)
  v2
   5
   4
   3
   2
   1
  Terms:
    'v2' (column 0)

DesignMatrix with shape (5, 4)
  Intercept  key_1[T.b]  key_2[T.zero]  key_1[T.b]:key_2[T.zero]
          1           0              1                         0
          1           0              0                         0
          1           1              0                         0
          1           1              1                         1
          1           0              0                         0
  Terms:
    'Intercept' (column 0)
    'key_1' (column 1)
    'key_2' (column 2)
    'key_1:key_2' (column 3)

## 二、statsmodels

**更多参考：** http://www.statsmodels.org

用于拟合多种模型，执行统计测试以及数据探索和可视化

In [144]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### （一）简单的OLS

In [105]:
# 生成实验数据
def dnorm(mean, variance, size=1):
    if isinstance(size, int):  # 判断是不是int类型
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# 
np.random.seed(12345)
n = 100
X = np.c_[dnorm(0, 0.4, size=n),
          dnorm(0, 0.6, size=n),
          dnorm(0, 0.2, size=n)]
X[:5]

eps = dnorm(0, 0.1, size=n)
beta = [0.1, 0.3, 0.5]
eps[:5]
beta

y = np.dot(X, beta) + eps
y[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

array([ 0.55252378, -0.44595899, -0.11961047, -0.1093581 ,  0.1201863 ])

[0.1, 0.3, 0.5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [108]:
# 添加截距列
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [115]:
# 拟合最小二乘法
model_1 = sm.OLS(y, X)  # 不带截距项
model_2 = sm.OLS(y, X_model)  # 带截距项

# 获取结果对象
result_1 = model_1.fit()
result_2 = model_2.fit()
result_1
result_2

# 获取参数
result_1.params
result_2.params

array([0.17826108, 0.22303962, 0.50095093])

array([0.03355856, 0.17614872, 0.22482596, 0.51480793])

In [118]:
# 获取详情
result_1.summary()
result_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Tue, 18 Aug 2020   Prob (F-statistic):           7.44e-12
Time:                        13:05:32   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.001       0.073       0.283
x2             0.2230      0.046      4.818      0.000       0.131       0.315
x3             0.5010      0.080      6.237      0.000       0.342       0.660
==============================================================================
Omnibus:                        4.662   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.097   Jarque-Bera (JB):                4.098
Skew:                           0.481   Prob(JB):                        0.129
Kurtosis:                       3.243   Cond. No.                         1.74
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     24.68
Date:                Tue, 18 Aug 2020   Prob (F-statistic):           6.37e-12
Time:                        13:05:32   Log-Likelihood:                -33.835
No. Observations:                 100   AIC:                             75.67
Df Residuals:                      96   BIC:                             86.09
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0336      0.035      0.952      0.343      -0.036       0.104
x1             0.1761      0.053      3.320      0.001       0.071       0.281
x2             0.2248      0.046      4.851      0.000       0.133       0.317
x3             0.5148      0.082      6.304      0.000       0.353       0.677
==============================================================================
Omnibus:                        4.504   Durbin-Watson:                   2.223
Prob(Omnibus):                  0.105   Jarque-Bera (JB):                3.957
Skew:                           0.475   Prob(JB):                        0.138
Kurtosis:                       3.222   Cond. No.                         2.38
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [138]:
# 当数据都在DataFrame中时, 使用patsy
df = pd.DataFrame(X_model, columns=['x0', 'x1', 'x2', 'x3'])
df['y'] = y
df[:5]

result_3 = sm.OLS(df['y'], df.iloc[:, :4]).fit()  # 直接利用df传参
result_3.params

result_4 = smf.ols('y ~ x1 + x2 + x3', data=df).fit()  # 利用patsy公式字符串传参
result_4.params

result_3.tvalues
result_4.tvalues

,x0,x1,x2,x3,y
0,1.0,-0.129468,-1.212753,0.504225,0.427863
1,1.0,0.302910,-0.435742,-0.254180,-0.673480
2,1.0,-0.328522,-0.025302,0.138351,-0.090878
3,1.0,-0.351475,-0.719605,-0.258215,-0.489494
4,1.0,1.243269,-0.373799,-0.522629,-0.128941


x0    0.033559
x1    0.176149
x2    0.224826
x3    0.514808
dtype: float64

Intercept    0.033559
x1           0.176149
x2           0.224826
x3           0.514808
dtype: float64

x0    0.952188
x1    3.319754
x2    4.850730
x3    6.303971
dtype: float64

Intercept    0.952188
x1           3.319754
x2           4.850730
x3           6.303971
dtype: float64

In [143]:
# 进行预测
result_3.predict(df.iloc[:5, :4])
result_4.predict(df.iloc[:5, :4])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

### （二）时间序列模型

In [145]:
pass

## 三、scikit-learn

**更多参考：** http://scikit-learn.org

机器学习库

In [148]:
train = pd.read_csv('.\\data_for_book\\chapter_13\\train.csv')
test = pd.read_csv('.\\data_for_book\\chapter_13\\test.csv')
train.head()
test.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### （一）缺失值处理

In [151]:
# 观察缺失值
train.isnull().sum()
test.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [152]:
# 填充Age，我们用训练集的中间值填充两个数据集的缺失值，只是作为案例这样做，现实中有更多的选择
in_v = train['Age'].median()
train['Age'] = train['Age'].fillna(in_v)
test['Age'] = test['Age'].fillna(in_v)

### （二）对Sex进行编码

In [156]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)
train['IsFemale'][:5]
test['IsFemale'][:5]

0    0
1    1
2    1
3    1
4    0
Name: IsFemale, dtype: int32

0    0
1    1
2    0
3    0
4    1
Name: IsFemale, dtype: int32

### （三）生成目标数据集

In [164]:
predictors = ['Pclass', 'IsFemale', 'Age']  # 我们只是选择了部分变量作为输入

X_train = train[predictors]
y_train = train['Survived']
X_test  = test[predictors]  # 测试集

X_train.head()
y_train.head()

,Pclass,IsFemale,Age
0,3,0,22.0
1,1,1,38.0
2,3,1,26.0
3,1,1,35.0
4,3,0,35.0


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### （四）建模

In [175]:
from sklearn.linear_model import LogisticRegression as LR

# 拟合
model = LR().fit(X=X_train, y=y_train)

# 预测
y_predict = model.predict(X_test)
y_predict[:5]

array([0, 0, 0, 0, 1], dtype=int64)

### （五）交叉验证

In [198]:
from sklearn.linear_model import LogisticRegressionCV as LRcv

model_cv = LRcv(10).fit(X_train, y_train)

# 手动交叉验证
from sklearn.model_selection import cross_val_score as cvs
model = LR(C=10)
scores_2 = cvs(model, X_train, y_train, cv=4)
scores_2

D:\software\Anaconda\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass Cs=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


array([0.77578475, 0.79820628, 0.77578475, 0.78828829])